# SetUp



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets torch pandas scikit-learn matplotlib seaborn

In [ ]:
!mkdir -p /content/drive/MyDrive/emotion_project

In [ ]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

The plan

✅ Suicide (always first - safety)

✅ Emotion (filter: positive = skip MH check)

✅ Mental Health (only if emotion negative)

# Load Dataset

In [ ]:
#SUICIDAL
suicide_df = pd.read_csv('Suicide_Detection.csv')

In [ ]:
# EMOTION
emotions_dataset = load_dataset("emotion")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
#MENTAL_HEALTH
mh_df = pd.read_csv('Combined Data.csv')

# Explore Raw Data

In [ ]:
#SUICIDE

print(f"\n📌 Shape: {suicide_df.shape}")
print(f"📌 Columns: {suicide_df.columns.tolist()}")
print(f"📌 Data types:\n{suicide_df.dtypes}")
print(f"\n🔎 Null values:\n{suicide_df.isnull().sum()}")
print(f"\n🔎 Duplicates: {suicide_df.duplicated().sum()}")
print(f"\n🏷️ Unique labels: {suicide_df['class'].unique()}")
print(f"\n🏷️ Label distribution:")
print(suicide_df['class'].value_counts())
print(f"\n📄 Sample rows:")
print(suicide_df.head(3))



📌 Shape: (232074, 3)
📌 Columns: ['Unnamed: 0', 'text', 'class']
📌 Data types:
Unnamed: 0     int64
text          object
class         object
dtype: object

🔎 Null values:
Unnamed: 0    0
text          0
class         0
dtype: int64

🔎 Duplicates: 0

🏷️ Unique labels: ['suicide' 'non-suicide']

🏷️ Label distribution:
class
suicide        116037
non-suicide    116037
Name: count, dtype: int64

📄 Sample rows:
   Unnamed: 0                                               text        class
0           2  Ex Wife Threatening SuicideRecently I left my ...      suicide
1           3  Am I weird I don't get affected by compliments...  non-suicide
2           4  Finally 2020 is almost over... So I can never ...  non-suicide


In [ ]:
#EMOTION
print(f"📌 Type: {type(emotions_dataset)}")
print(f"📌 Keys: {emotions_dataset.keys()}")

print(f"\n🔹 TRAIN split:")
print(f"   Rows: {len(emotions_dataset['train'])}")
print(f"   Columns: {emotions_dataset['train'].column_names}")

print(f"\n🔹 VALIDATION split:")
print(f"   Rows: {len(emotions_dataset['validation'])}")
print(f"   Columns: {emotions_dataset['validation'].column_names}")

print(f"\n🔹 TEST split:")
print(f"   Rows: {len(emotions_dataset['test'])}")
print(f"   Columns: {emotions_dataset['test'].column_names}")

print("TRAIN split exploration:")
emo_train_df = pd.DataFrame(emotions_dataset['train'])

print(f"\n🏷️ Label values in train: {emo_train_df['label'].unique()}")
print(f"\n🏷️ Label distribution:")
print(emo_train_df['label'].value_counts().sort_index())

print(f"\n🏷️ Label mapping:")
label_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
for i, name in enumerate(label_names):
    count = (emo_train_df['label'] == i).sum()
    print(f"   {i}: {name} ({count} samples)")

print(f"\n🔎 Null values in train:\n{emo_train_df.isnull().sum()}")
print(f"\n🔎 Duplicates in train: {emo_train_df.duplicated().sum()}")

print(emo_train_df.head())



📌 Type: <class 'datasets.dataset_dict.DatasetDict'>
📌 Keys: dict_keys(['train', 'validation', 'test'])

🔹 TRAIN split:
   Rows: 16000
   Columns: ['text', 'label']

🔹 VALIDATION split:
   Rows: 2000
   Columns: ['text', 'label']

🔹 TEST split:
   Rows: 2000
   Columns: ['text', 'label']
TRAIN split exploration:

🏷️ Label values in train: [0 3 2 5 4 1]

🏷️ Label distribution:
label
0    4666
1    5362
2    1304
3    2159
4    1937
5     572
Name: count, dtype: int64

🏷️ Label mapping:
   0: sadness (4666 samples)
   1: joy (5362 samples)
   2: love (1304 samples)
   3: anger (2159 samples)
   4: fear (1937 samples)
   5: surprise (572 samples)

🔎 Null values in train:
text     0
label    0
dtype: int64

🔎 Duplicates in train: 1
                                                text  label
0                            i didnt feel humiliated      0
1  i can go from feeling so hopeless to so damned...      0
2   im grabbing a minute to post i feel greedy wrong      3
3  i am ever feeling no

In [ ]:
#MENTAL_HEALTH

print(f"\n📌 Shape: {mh_df.shape}")
print(f"📌 Columns: {mh_df.columns.tolist()}")
print(f"📌 Data types:\n{mh_df.dtypes}")

print(f"\n🔎 Null values:\n{mh_df.isnull().sum()}")
print(f"\n🔎 Duplicates: {mh_df.duplicated().sum()}")

print(f"\n🏷️ Unique labels: {mh_df['status'].unique()}")
print(f"\n🏷️ Label distribution:")
print(mh_df['status'].value_counts())

if 'Suicidal' in mh_df['status'].values:
    suicidal_count = (mh_df['status'] == 'Suicidal').sum()
    print(f"\n⚠️ 'Suicidal' label found: {suicidal_count} samples")
    print("   (Will be removed during preprocessing) ")

print(f"\n📄 Sample rows:")
print(mh_df.head(3))


📌 Shape: (53043, 3)
📌 Columns: ['Unnamed: 0', 'statement', 'status']
📌 Data types:
Unnamed: 0     int64
statement     object
status        object
dtype: object

🔎 Null values:
Unnamed: 0      0
statement     362
status          0
dtype: int64

🔎 Duplicates: 0

🏷️ Unique labels: ['Anxiety' 'Normal' 'Depression' 'Suicidal' 'Stress' 'Bipolar'
 'Personality disorder']

🏷️ Label distribution:
status
Normal                  16351
Depression              15404
Suicidal                10653
Anxiety                  3888
Bipolar                  2877
Stress                   2669
Personality disorder     1201
Name: count, dtype: int64

⚠️ 'Suicidal' label found: 10653 samples
   (Will be removed during preprocessing) 

📄 Sample rows:
   Unnamed: 0                                          statement   status
0           0                                         oh my gosh  Anxiety
1           1  trouble sleeping, confused mind, restless hear...  Anxiety
2           2  All wrong, back off dear, f

In [ ]:
# DATA EXPLORATION - Add length columns for analysis

print("="*60)
print("📏 ADDING LENGTH COLUMNS FOR ANALYSIS")
print("="*60)

# Suicide
suicide_df['word_count'] = suicide_df['text'].str.split().str.len()
print(f"\n✅ Suicide: Added 'word_count' column")

# Emotion
emo_train_df['word_count'] = emo_train_df['text'].str.split().str.len()
print(f"✅ Emotion: Added 'word_count' column")

# Mental Health
mh_df['word_count'] = mh_df['statement'].str.split().str.len()
print(f"✅ Mental Health: Added 'word_count' column")

# Check stats
print("\n" + "="*60)
print("📊 LENGTH STATISTICS")
print("="*60)

print(f"\nSuicide:")
print(f"  Max: {suicide_df['word_count'].max()} | Mean: {suicide_df['word_count'].mean():.0f} | Median: {suicide_df['word_count'].median():.0f}")
print(f"  Over 400 words: {(suicide_df['word_count'] > 400).sum()} ({(suicide_df['word_count'] > 400).sum()/len(suicide_df)*100:.2f}%)")

print(f"\nEmotion:")
print(f"  Max: {emo_train_df['word_count'].max()} | Mean: {emo_train_df['word_count'].mean():.0f} | Median: {emo_train_df['word_count'].median():.0f}")
print(f"  Over 400 words: {(emo_train_df['word_count'] > 400).sum()} ({(emo_train_df['word_count'] > 400).sum()/len(emo_train_df)*100:.2f}%)")

print(f"\nMental Health:")
print(f"  Max: {mh_df['word_count'].max()} | Mean: {mh_df['word_count'].mean():.0f} | Median: {mh_df['word_count'].median():.0f}")
print(f"  Over 400 words: {(mh_df['word_count'] > 400).sum()} ({(mh_df['word_count'] > 400).sum()/len(mh_df)*100:.2f}%)")

print("\n" + "="*60)

📏 ADDING LENGTH COLUMNS FOR ANALYSIS

✅ Suicide: Added 'word_count' column
✅ Emotion: Added 'word_count' column
✅ Mental Health: Added 'word_count' column

📊 LENGTH STATISTICS

Suicide:
  Max: 9684 | Mean: 132 | Median: 60
  Over 400 words: 15820 (6.82%)

Emotion:
  Max: 66 | Mean: 19 | Median: 17
  Over 400 words: 0 (0.00%)

Mental Health:
  Max: 6300.0 | Mean: 113 | Median: 62
  Over 400 words: 2564 (4.83%)



# Data Preprocessing

In [ ]:
# DATA PREPROCESSING - Remove temporary analysis columns

print("="*60)
print("🧹 REMOVING TEMPORARY COLUMNS")
print("="*60)

# Before
print(f"\nBefore:")
print(f"  Suicide columns: {suicide_df.columns.tolist()}")
print(f"  Emotion columns: {emo_train_df.columns.tolist()}")
print(f"  Mental Health columns: {mh_df.columns.tolist()}")

# Remove word_count
suicide_df = suicide_df.drop(columns=['word_count'])
emo_train_df = emo_train_df.drop(columns=['word_count'])
mh_df = mh_df.drop(columns=['word_count'])

# After
print(f"\nAfter:")
print(f"  Suicide columns: {suicide_df.columns.tolist()}")
print(f"  Emotion columns: {emo_train_df.columns.tolist()}")
print(f"  Mental Health columns: {mh_df.columns.tolist()}")

print("\n✅ Temporary columns removed!")
print("="*60)

🧹 REMOVING TEMPORARY COLUMNS

Before:
  Suicide columns: ['Unnamed: 0', 'text', 'class', 'word_count']
  Emotion columns: ['text', 'label', 'word_count']
  Mental Health columns: ['Unnamed: 0', 'statement', 'status', 'word_count']

After:
  Suicide columns: ['Unnamed: 0', 'text', 'class']
  Emotion columns: ['text', 'label']
  Mental Health columns: ['Unnamed: 0', 'statement', 'status']

✅ Temporary columns removed!


In [ ]:
# Suicidal

print("\n1️⃣ Removing 'Unnamed: 0' column")
suicide_clean = suicide_df[['text', 'class']].copy()
print(f"   Columns: {suicide_clean.columns.tolist()}")

print("\n2️⃣ Renaming 'class' → 'label'")
suicide_clean = suicide_clean.rename(columns={'class': 'label'})
print(f"   Columns: {suicide_clean.columns.tolist()}")

suicide_clean = suicide_clean.reset_index(drop=True)
print(f"\n✅ Cleaned: {len(suicide_clean):,} rows")
print(f"   Labels: {suicide_clean['label'].unique().tolist()}")

print("🔪 SPLITTING (80% train / 10% val / 10% test)")

sui_train, sui_temp = train_test_split(
    suicide_clean,
    test_size=0.2,
    random_state=42,
    stratify=suicide_clean['label']
)

sui_val, sui_test = train_test_split(
    sui_temp,
    test_size=0.5,
    random_state=42,
    stratify=sui_temp['label']
)

print(f"\n✅ SPLITS:")
print(f"   sui_train: {len(sui_train):,} ({len(sui_train)/len(suicide_clean)*100:.1f}%)")
print(f"   sui_val:   {len(sui_val):,} ({len(sui_val)/len(suicide_clean)*100:.1f}%)")
print(f"   sui_test:  {len(sui_test):,} ({len(sui_test)/len(suicide_clean)*100:.1f}%)")

print(f"\n📊 LABEL DISTRIBUTION:")
print(f"\n   Train:")
print(f"   {dict(sui_train['label'].value_counts())}")
print(f"\n   Val:")
print(f"   {dict(sui_val['label'].value_counts())}")
print(f"\n   Test:")
print(f"   {dict(sui_test['label'].value_counts())}")

sui_train.to_csv('sui_train.csv', index=False)
sui_val.to_csv('sui_val.csv', index=False)
sui_test.to_csv('sui_test.csv', index=False)


1️⃣ Removing 'Unnamed: 0' column
   Columns: ['text', 'class']

2️⃣ Renaming 'class' → 'label'
   Columns: ['text', 'label']

✅ Cleaned: 232,074 rows
   Labels: ['suicide', 'non-suicide']
🔪 SPLITTING (80% train / 10% val / 10% test)

✅ SPLITS:
   sui_train: 185,659 (80.0%)
   sui_val:   23,207 (10.0%)
   sui_test:  23,208 (10.0%)

📊 LABEL DISTRIBUTION:

   Train:
   {'suicide': np.int64(92830), 'non-suicide': np.int64(92829)}

   Val:
   {'non-suicide': np.int64(11604), 'suicide': np.int64(11603)}

   Test:
   {'suicide': np.int64(11604), 'non-suicide': np.int64(11604)}


In [ ]:
#Emotion

emo_train_df = pd.DataFrame(emotions_dataset['train'])
emo_val_df = pd.DataFrame(emotions_dataset['validation'])
emo_test_df = pd.DataFrame(emotions_dataset['test'])

print(f"   Train: {len(emo_train_df):,} rows")
print(f"   Val:   {len(emo_val_df):,} rows")
print(f"   Test:  {len(emo_test_df):,} rows")

label_map = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}

emo_train_df['label'] = emo_train_df['label'].map(label_map)
emo_val_df['label'] = emo_val_df['label'].map(label_map)
emo_test_df['label'] = emo_test_df['label'].map(label_map)

print(f"   Mapped to: {sorted(emo_train_df['label'].unique().tolist())}")

print("\nRemoving nulls from all splits")
train_nulls_before = len(emo_train_df)
val_nulls_before = len(emo_val_df)
test_nulls_before = len(emo_test_df)

emo_train_df = emo_train_df.dropna()
emo_val_df = emo_val_df.dropna()
emo_test_df = emo_test_df.dropna()

print(f"   Train: Removed {train_nulls_before - len(emo_train_df)} nulls")
print(f"   Val:   Removed {val_nulls_before - len(emo_val_df)} nulls")
print(f"   Test:  Removed {test_nulls_before - len(emo_test_df)} nulls")

print("\nRemoving duplicates from all splits")
train_dup_before = len(emo_train_df)
val_dup_before = len(emo_val_df)
test_dup_before = len(emo_test_df)

emo_train_df = emo_train_df.drop_duplicates()
emo_val_df = emo_val_df.drop_duplicates()
emo_test_df = emo_test_df.drop_duplicates()

print(f"   Train: Removed {train_dup_before - len(emo_train_df)} duplicates")
print(f"   Val:   Removed {val_dup_before - len(emo_val_df)} duplicates")
print(f"   Test:  Removed {test_dup_before - len(emo_test_df)} duplicates")

emo_train_df = emo_train_df[['text', 'label']]
emo_val_df = emo_val_df[['text', 'label']]
emo_test_df = emo_test_df[['text', 'label']]

emo_train = emo_train_df
emo_val = emo_val_df
emo_test = emo_test_df

print(f"\n📊 LABEL DISTRIBUTION (for info only):")
print(f"\n   Train:")
print(emo_train['label'].value_counts().sort_index().to_dict())
print(f"\n   Val:")
print(emo_val['label'].value_counts().sort_index().to_dict())
print(f"\n   Test:")
print(emo_test['label'].value_counts().sort_index().to_dict())

emo_train.to_csv('emo_train.csv', index=False)
emo_val.to_csv('emo_val.csv', index=False)
emo_test.to_csv('emo_test.csv', index=False)

   Train: 16,000 rows
   Val:   2,000 rows
   Test:  2,000 rows
   Mapped to: ['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']

Removing nulls from all splits
   Train: Removed 0 nulls
   Val:   Removed 0 nulls
   Test:  Removed 0 nulls

Removing duplicates from all splits
   Train: Removed 1 duplicates
   Val:   Removed 0 duplicates
   Test:  Removed 0 duplicates

📊 LABEL DISTRIBUTION (for info only):

   Train:
{'anger': 2159, 'fear': 1937, 'joy': 5361, 'love': 1304, 'sadness': 4666, 'surprise': 572}

   Val:
{'anger': 275, 'fear': 212, 'joy': 704, 'love': 178, 'sadness': 550, 'surprise': 81}

   Test:
{'anger': 275, 'fear': 224, 'joy': 695, 'love': 159, 'sadness': 581, 'surprise': 66}


In [ ]:
#Mental Health

print("\n1️Removing 'Unnamed: 0' column")
mh_clean = mh_df[['statement', 'status']].copy()
print(f"   Columns: {mh_clean.columns.tolist()}")

print("\nRenaming columns")
mh_clean = mh_clean.rename(columns={'statement': 'text', 'status': 'label'})
print(f"   Columns: {mh_clean.columns.tolist()}")

print("\nRemoving nulls")
before = len(mh_clean)
mh_clean = mh_clean.dropna()
after = len(mh_clean)
print(f"   Removed: {before - after} rows with nulls")

print("\nRemoving 'Suicidal' label")
before = len(mh_clean)
mh_clean = mh_clean[mh_clean['label'] != 'Suicidal']
after = len(mh_clean)
print(f"   Removed: {before - after} 'Suicidal' samples")

print(f"\n✅ Remaining labels: {sorted(mh_clean['label'].unique().tolist())}")
print(f"\n📊 Label distribution:")
print(mh_clean['label'].value_counts().to_dict())

print("\nRemoving duplicates")
before = len(mh_clean)
mh_clean = mh_clean.drop_duplicates()
after = len(mh_clean)
print(f"   Removed: {before - after} duplicates")

mh_clean = mh_clean.reset_index(drop=True)
print(f"\n✅ Cleaned dataset: {len(mh_clean):,} rows")
print("🔪 SPLITTING (80% train / 10% val / 10% test)")
mh_train, mh_temp = train_test_split(
    mh_clean,
    test_size=0.2,
    random_state=42,
    stratify=mh_clean['label']
)
mh_val, mh_test = train_test_split(
    mh_temp,
    test_size=0.5,
    random_state=42,
    stratify=mh_temp['label']
)
print(f"\n✅ SPLITS:")
print(f"   mh_train: {len(mh_train):,} ({len(mh_train)/len(mh_clean)*100:.1f}%)")
print(f"   mh_val:   {len(mh_val):,} ({len(mh_val)/len(mh_clean)*100:.1f}%)")
print(f"   mh_test:  {len(mh_test):,} ({len(mh_test)/len(mh_clean)*100:.1f}%)")

print(f"\n📊 LABEL DISTRIBUTION:")
print(f"\n   Train:")
print(mh_train['label'].value_counts().to_dict())
print(f"\n   Val:")
print(mh_val['label'].value_counts().to_dict())
print(f"\n   Test:")
print(mh_test['label'].value_counts().to_dict())

mh_train.to_csv('mh_train.csv', index=False)
mh_val.to_csv('mh_val.csv', index=False)
mh_test.to_csv('mh_test.csv', index=False)


1️Removing 'Unnamed: 0' column
   Columns: ['statement', 'status']

Renaming columns
   Columns: ['text', 'label']

Removing nulls
   Removed: 362 rows with nulls

Removing 'Suicidal' label
   Removed: 10652 'Suicidal' samples

✅ Remaining labels: ['Anxiety', 'Bipolar', 'Depression', 'Normal', 'Personality disorder', 'Stress']

📊 Label distribution:
{'Normal': 16343, 'Depression': 15404, 'Anxiety': 3841, 'Bipolar': 2777, 'Stress': 2587, 'Personality disorder': 1077}

Removing duplicates
   Removed: 1580 duplicates

✅ Cleaned dataset: 40,449 rows
🔪 SPLITTING (80% train / 10% val / 10% test)

✅ SPLITS:
   mh_train: 32,359 (80.0%)
   mh_val:   4,045 (10.0%)
   mh_test:  4,045 (10.0%)

📊 LABEL DISTRIBUTION:

   Train:
{'Normal': 12832, 'Depression': 12075, 'Anxiety': 2898, 'Bipolar': 2001, 'Stress': 1837, 'Personality disorder': 716}

   Val:
{'Normal': 1604, 'Depression': 1510, 'Anxiety': 362, 'Bipolar': 250, 'Stress': 230, 'Personality disorder': 89}

   Test:
{'Normal': 1604, 'Depressi

In [ ]:
# Saving datasets to drive
save_path = '/content/drive/MyDrive/emotion_project'
print(f"\n📁 Save location: {save_path}")

# Save Suicide datasets
print("\n1️⃣ Saving Suicide datasets...")
sui_train.to_csv(f'{save_path}/sui_train.csv', index=False)
sui_val.to_csv(f'{save_path}/sui_val.csv', index=False)
sui_test.to_csv(f'{save_path}/sui_test.csv', index=False)
print("   ✅ sui_train.csv, sui_val.csv, sui_test.csv")

# Save Emotion datasets
print("\n2️⃣ Saving Emotion datasets...")
emo_train.to_csv(f'{save_path}/emo_train.csv', index=False)
emo_val.to_csv(f'{save_path}/emo_val.csv', index=False)
emo_test.to_csv(f'{save_path}/emo_test.csv', index=False)
print("   ✅ emo_train.csv, emo_val.csv, emo_test.csv")

# Save Mental Health datasets
print("\n3️⃣ Saving Mental Health datasets...")
mh_train.to_csv(f'{save_path}/mh_train.csv', index=False)
mh_val.to_csv(f'{save_path}/mh_val.csv', index=False)
mh_test.to_csv(f'{save_path}/mh_test.csv', index=False)
print("   ✅ mh_train.csv, mh_val.csv, mh_test.csv")



📁 Save location: /content/drive/MyDrive/emotion_project

1️⃣ Saving Suicide datasets...
   ✅ sui_train.csv, sui_val.csv, sui_test.csv

2️⃣ Saving Emotion datasets...
   ✅ emo_train.csv, emo_val.csv, emo_test.csv

3️⃣ Saving Mental Health datasets...
   ✅ mh_train.csv, mh_val.csv, mh_test.csv


# Switching to GPU

In [ ]:
from IPython.display import Javascript
display(Javascript('''
function KeepClicking(){
   console.log("Clicking");
   document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking, 60000)
'''))
print("✅ Session keeper active!")

<IPython.core.display.Javascript object>

✅ Session keeper active!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
if torch.cuda.is_available():
    print("\n✅ GPU is active!")
    print(f"   Device: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("\n❌ WARNING: GPU not detected!")


❌ WARNING: GPU not detected!


In [ ]:
import pandas as pd

save_path = '/content/drive/MyDrive/emotion_project'

# Load Suicide datasets
print("\n1️⃣ Loading Suicide Detection datasets...")
sui_train = pd.read_csv(f'{save_path}/sui_train.csv')
sui_val = pd.read_csv(f'{save_path}/sui_val.csv')
sui_test = pd.read_csv(f'{save_path}/sui_test.csv')

print(f"   ✅ sui_train: {len(sui_train):,} rows")
print(f"   ✅ sui_val:   {len(sui_val):,} rows")
print(f"   ✅ sui_test:  {len(sui_test):,} rows")

# Load Emotion datasets
print("\n2️⃣ Loading Emotion Detection datasets...")
emo_train = pd.read_csv(f'{save_path}/emo_train.csv')
emo_val = pd.read_csv(f'{save_path}/emo_val.csv')
emo_test = pd.read_csv(f'{save_path}/emo_test.csv')

print(f"   ✅ emo_train: {len(emo_train):,} rows")
print(f"   ✅ emo_val:   {len(emo_val):,} rows")
print(f"   ✅ emo_test:  {len(emo_test):,} rows")

# Load Mental Health datasets
print("\n3️⃣ Loading Mental Health Detection datasets...")
mh_train = pd.read_csv(f'{save_path}/mh_train.csv')
mh_val = pd.read_csv(f'{save_path}/mh_val.csv')
mh_test = pd.read_csv(f'{save_path}/mh_test.csv')

print(f"   ✅ mh_train:  {len(mh_train):,} rows")
print(f"   ✅ mh_val:    {len(mh_val):,} rows")
print(f"   ✅ mh_test:   {len(mh_test):,} rows")


1️⃣ Loading Suicide Detection datasets...
   ✅ sui_train: 185,659 rows
   ✅ sui_val:   23,207 rows
   ✅ sui_test:  23,208 rows

2️⃣ Loading Emotion Detection datasets...
   ✅ emo_train: 15,999 rows
   ✅ emo_val:   2,000 rows
   ✅ emo_test:  2,000 rows

3️⃣ Loading Mental Health Detection datasets...
   ✅ mh_train:  32,359 rows
   ✅ mh_val:    4,045 rows
   ✅ mh_test:   4,045 rows


# Training Suicidal Model

In [ ]:
#Install and import

!pip install transformers datasets accelerate -q

import pandas as pd
import numpy as np
import torch
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

print("\n✅ Libraries installed!")


✅ Libraries installed!


In [ ]:
#Encode Labels

label_list = sorted(sui_train['label'].unique())
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

print(f"\nLabel mapping:")
for label, idx in label2id.items():
    print(f"  {idx}: {label}")

# Add encoded labels
sui_train['label_id'] = sui_train['label'].map(label2id)
sui_val['label_id'] = sui_val['label'].map(label2id)
sui_test['label_id'] = sui_test['label'].map(label2id)

print(f"\n✅ Labels encoded!")


Label mapping:
  0: non-suicide
  1: suicide

✅ Labels encoded!


In [ ]:
#Tokenize

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
print("✅ Tokenizer loaded")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=512
    )

# Convert to Dataset
train_dataset = Dataset.from_pandas(sui_train[['text', 'label_id']])
val_dataset = Dataset.from_pandas(sui_val[['text', 'label_id']])
test_dataset = Dataset.from_pandas(sui_test[['text', 'label_id']])

print("\n⚙️ Tokenizing (watch the progress bar)...\n")

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True, batch_size=1000)
val_dataset = val_dataset.map(tokenize_function, batched=True, batch_size=1000)
test_dataset = test_dataset.map(tokenize_function, batched=True, batch_size=1000)

# Prepare for training
train_dataset = train_dataset.rename_column('label_id', 'labels')
val_dataset = val_dataset.rename_column('label_id', 'labels')
test_dataset = test_dataset.rename_column('label_id', 'labels')

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print(f"\n✅ Tokenization complete!")

✅ Tokenizer loaded

⚙️ Tokenizing (watch the progress bar)...



Map:   0%|          | 0/185659 [00:00<?, ? examples/s]

Map:   0%|          | 0/23207 [00:00<?, ? examples/s]

Map:   0%|          | 0/23208 [00:00<?, ? examples/s]


✅ Tokenization complete!


In [ ]:
# Loading Model

device = torch.device('cuda')
model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)
model.to(device)

print(f"\n✅ Model loaded on GPU!")
print(f"   Parameters: {model.num_parameters():,}")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



✅ Model loaded on GPU!
   Parameters: 124,647,170


In [ ]:
# Defining metrics

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted'
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

print("✅ Metrics defined")

✅ Metrics defined


In [ ]:
# Training Configuration

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/emotion_project/suicide_model',

    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,

    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,

    eval_strategy='steps',
    eval_steps=500,
    save_strategy='steps',
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,

    logging_steps=100,
    fp16=True,
    report_to='none'
)

print(f"✅ Config set")
print(f"   Epochs: 3")
print(f"   Estimated time: 1.5-2 hours")

✅ Config set
   Epochs: 3
   Estimated time: 1.5-2 hours


In [ ]:
#Training

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

import os
checkpoint_dir = '/content/drive/MyDrive/emotion_project/suicide_model'
has_checkpoint = False

if os.path.exists(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith('checkpoint-')]
    if checkpoints:
        has_checkpoint = True
        print(f"✅ Found checkpoint: {sorted(checkpoints)[-1]}")
        print("   Resuming from checkpoint...\n")

if has_checkpoint:
    train_result = trainer.train(resume_from_checkpoint=True)
else:
    train_result = trainer.train()

print(f"\nTime: {train_result.metrics['train_runtime']/60:.1f} minutes")
print(f"Loss: {train_result.training_loss:.4f}")


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
10000,0.028700,0.039772,0.992761,0.992807,0.992761,0.992761
10500,0.030400,0.037140,0.991382,0.991485,0.991382,0.991381
11000,0.052000,0.034556,0.992114,0.992168,0.992114,0.992114



Time: 18.1 minutes
Loss: 0.0051


In [ ]:
# Evaluation
eval_results = trainer.evaluate()
print(f"\n✅ Validation:")
print(f"   Accuracy:  {eval_results['eval_accuracy']:.4f} ({eval_results['eval_accuracy']*100:.2f}%)")
print(f"   Precision: {eval_results['eval_precision']:.4f}")
print(f"   Recall:    {eval_results['eval_recall']:.4f}")
print(f"   F1 Score:  {eval_results['eval_f1']:.4f}")

# Test
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids
test_acc = accuracy_score(labels, preds)

print(f"\n✅ Test:")
print(f"   Accuracy: {test_acc*100:.2f}%")

# Confusion Matrix
cm = confusion_matrix(labels, preds)
print(f"\n📊 Confusion Matrix:")
print(f"              Predicted")
print(f"           Non-suicide  Suicide")
print(f"Actual")
print(f"Non-suicide  {cm[0][0]:<12} {cm[0][1]:<12}")
print(f"Suicide      {cm[1][0]:<12} {cm[1][1]:<12}")


✅ Validation:
   Accuracy:  0.9923 (99.23%)
   Precision: 0.9924
   Recall:    0.9923
   F1 Score:  0.9923

✅ Test:
   Accuracy: 99.22%

📊 Confusion Matrix:
              Predicted
           Non-suicide  Suicide
Actual
Non-suicide  11583        21          
Suicide      161          11443       


In [ ]:
print(f"   Val loss:  {eval_results['eval_loss']:.4f}")

   Val loss:  0.0267


In [ ]:
# Save suicideal model to drive

model_path = '/content/drive/MyDrive/emotion_project/suicide_model_final'
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

import json
with open(f'{model_path}/label_mapping.json', 'w') as f:
    json.dump({'label2id': label2id, 'id2label': id2label}, f)

print(f"\n✅ Saved to: {model_path}")


✅ Saved to: /content/drive/MyDrive/emotion_project/suicide_model_final


# Training Emotional Model

In [ ]:
!pip install transformers datasets accelerate -q

import pandas as pd
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

print("\n✅ Libraries ready!")


✅ Libraries ready!


In [ ]:
# Encoding labels

# Create mappings
label_list = sorted(emo_train['label'].unique())
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

print(f"\nLabel mapping:")
for label, idx in label2id.items():
    count = (emo_train['label'] == label).sum()
    print(f"  {idx}: {label} ({count:,} samples)")

# Encode
emo_train['label_id'] = emo_train['label'].map(label2id)
emo_val['label_id'] = emo_val['label'].map(label2id)
emo_test['label_id'] = emo_test['label'].map(label2id)

print(f"\n✅ Labels encoded!")


Label mapping:
  0: anger (2,159 samples)
  1: fear (1,937 samples)
  2: joy (5,361 samples)
  3: love (1,304 samples)
  4: sadness (4,666 samples)
  5: surprise (572 samples)

✅ Labels encoded!


In [ ]:
#Tokenize

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
print("✅ Tokenizer loaded")

def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=512
    )

# Convert to Dataset
train_dataset = Dataset.from_pandas(emo_train[['text', 'label_id']])
val_dataset = Dataset.from_pandas(emo_val[['text', 'label_id']])
test_dataset = Dataset.from_pandas(emo_test[['text', 'label_id']])

print("\n⚙️ Tokenizing...\n")

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True, batch_size=1000)
val_dataset = val_dataset.map(tokenize_function, batched=True, batch_size=1000)
test_dataset = test_dataset.map(tokenize_function, batched=True, batch_size=1000)

# Prepare
train_dataset = train_dataset.rename_column('label_id', 'labels')
val_dataset = val_dataset.rename_column('label_id', 'labels')
test_dataset = test_dataset.rename_column('label_id', 'labels')

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print(f"\n✅ Tokenization complete!")

✅ Tokenizer loaded

⚙️ Tokenizing...



Map:   0%|          | 0/15999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]


✅ Tokenization complete!


In [ ]:
#Load model

device = torch.device('cuda')
model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=6,
    id2label=id2label,
    label2id=label2id
)
model.to(device)

print(f"\n✅ Model loaded!")
print(f"   Num labels: 6")
print(f"   Parameters: {model.num_parameters():,}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



✅ Model loaded!
   Num labels: 6
   Parameters: 124,650,246


In [ ]:
# defining metrics

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted'
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

print("✅ Metrics defined")

✅ Metrics defined


In [ ]:
# Training Config

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/emotion_project/emotion_model',

    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,

    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,

    eval_strategy='steps',
    eval_steps=200,
    save_strategy='steps',
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',

    logging_steps=50,
    fp16=True,
    report_to='none'
)

In [ ]:
#Training

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

import os
checkpoint_dir = '/content/drive/MyDrive/emotion_project/emotion_model'
has_checkpoint = False

if os.path.exists(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith('checkpoint-')]
    if checkpoints:
        has_checkpoint = True
        print(f"\n✅ Found checkpoint: {sorted(checkpoints)[-1]}")
        print("   Resuming from checkpoint...")

if has_checkpoint:
    train_result = trainer.train(resume_from_checkpoint=True)
else:
    train_result = trainer.train()

print(f"Time: {train_result.metrics['train_runtime']/60:.1f} minutes")
print(f"Loss: {train_result.training_loss:.4f}")


✅ Found checkpoint: checkpoint-1600
   Resuming from checkpoint...


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1800,0.131500,0.199238,0.931000,0.932099,0.931000,0.931326
2000,0.134600,0.187806,0.935000,0.935412,0.935000,0.934776
2200,0.160100,0.165420,0.934500,0.935862,0.934500,0.934830
2400,0.100400,0.194787,0.932000,0.935774,0.932000,0.932405
2600,0.116200,0.167291,0.937500,0.937671,0.937500,0.937518
2800,0.142400,0.166526,0.933000,0.935011,0.933000,0.933514
3000,0.111200,0.162565,0.934000,0.936093,0.934000,0.934634


Time: 12.6 minutes
Loss: 0.0657


In [ ]:
#Evaluation

# Validation
eval_results = trainer.evaluate()
print(f"\n✅ Validation:")
print(f"   Accuracy: {eval_results['eval_accuracy']*100:.2f}%")
print(f"   Precision: {eval_results['eval_precision']:.4f}")
print(f"   Recall: {eval_results['eval_recall']:.4f}")
print(f"   F1: {eval_results['eval_f1']:.4f}")

# Test
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids
test_acc = accuracy_score(labels, preds)

print(f"\n✅ Test:")
print(f"   Accuracy: {test_acc*100:.2f}%")

# Confusion Matrix
cm = confusion_matrix(labels, preds)
print(f"\n📊 Confusion Matrix:")
print(f"\nLabels: {list(id2label.values())}")
print(cm)


✅ Validation:
   Accuracy: 93.75%
   Precision: 0.9377
   Recall: 0.9375
   F1: 0.9375

✅ Test:
   Accuracy: 93.30%

📊 Confusion Matrix:

Labels: ['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']
[[256  12   2   0   5   0]
 [  0 206   0   0   9   9]
 [  1   1 662  22   4   5]
 [  1   0  26 132   0   0]
 [ 15   3   2   0 561   0]
 [  0  14   0   0   3  49]]


In [ ]:
print(f"   Validation Loss: {eval_results['eval_loss']:.4f}")

   Validation Loss: 0.1673


In [ ]:
#Saving

model_path = '/content/drive/MyDrive/emotion_project/emotion_model_final'
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

import json
with open(f'{model_path}/label_mapping.json', 'w') as f:
    json.dump({'label2id': label2id, 'id2label': id2label}, f, indent=2)

print(f"\n✅ Saved to: {model_path}")
print("\n🎉 MODEL 2 COMPLETE!")


✅ Saved to: /content/drive/MyDrive/emotion_project/emotion_model_final

🎉 MODEL 2 COMPLETE!


# Mental Model Training

In [ ]:
# Setup

import pandas as pd
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

print("\n✅ Ready!")

ImportError: cannot import name 'PreTrainedModel' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

In [ ]:
#encoding labels

# Create mappings
label_list = sorted(mh_train['label'].unique())
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

print(f"\nLabel mapping:")
for label, idx in label2id.items():
    count = (mh_train['label'] == label).sum()
    print(f"  {idx}: {label} ({count:,} samples)")

# Encode
mh_train['label_id'] = mh_train['label'].map(label2id)
mh_val['label_id'] = mh_val['label'].map(label2id)
mh_test['label_id'] = mh_test['label'].map(label2id)

print(f"\n✅ Labels encoded!")

In [ ]:
# tokenizing

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
print("✅ Tokenizer loaded")

def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=512
    )

# Convert to Dataset
train_dataset = Dataset.from_pandas(mh_train[['text', 'label_id']])
val_dataset = Dataset.from_pandas(mh_val[['text', 'label_id']])
test_dataset = Dataset.from_pandas(mh_test[['text', 'label_id']])

print("\n⚙️ Tokenizing...\n")

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True, batch_size=1000)
val_dataset = val_dataset.map(tokenize_function, batched=True, batch_size=1000)
test_dataset = test_dataset.map(tokenize_function, batched=True, batch_size=1000)

# Prepare
train_dataset = train_dataset.rename_column('label_id', 'labels')
val_dataset = val_dataset.rename_column('label_id', 'labels')
test_dataset = test_dataset.rename_column('label_id', 'labels')

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print(f"\n✅ Tokenization complete!")

In [ ]:
# load model


device = torch.device('cuda')
model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=6,
    id2label=id2label,
    label2id=label2id
)
model.to(device)

print(f"\n✅ Model loaded!")
print(f"   Num labels: 6")
print(f"   Parameters: {model.num_parameters():,}")

In [ ]:
# defining metrics

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted'
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

print("✅ Metrics defined")

In [ ]:
# training config

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/emotion_project/mh_model',

    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,

    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,

    eval_strategy='steps',
    eval_steps=300,
    save_strategy='steps',
    save_steps=300,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',

    logging_steps=100,
    fp16=True,
    report_to='none'
)


In [ ]:
# training

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Check for checkpoints
import os
checkpoint_dir = '/content/drive/MyDrive/emotion_project/mh_model'
has_checkpoint = False

if os.path.exists(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith('checkpoint-')]
    if checkpoints:
        has_checkpoint = True
        print(f"✅ Found checkpoint: {sorted(checkpoints)[-1]}")
        print("   Resuming from checkpoint...\n")

if has_checkpoint:
    train_result = trainer.train(resume_from_checkpoint=True)
else:
    train_result = trainer.train()

print(f"Time: {train_result.metrics['train_runtime']/60:.1f} minutes")
print(f"Loss: {train_result.training_loss:.4f}")

In [ ]:
#evaluation
# Validation
eval_results = trainer.evaluate()
print(f"\n✅ Validation:")
print(f"   Accuracy: {eval_results['eval_accuracy']*100:.2f}%")
print(f"   Precision: {eval_results['eval_precision']:.4f}")
print(f"   Recall: {eval_results['eval_recall']:.4f}")
print(f"   F1: {eval_results['eval_f1']:.4f}")
print(f"   Val Loss: {eval_results['eval_loss']:.4f}")

# Test
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids
test_acc = accuracy_score(labels, preds)

print(f"\n✅ Test:")
print(f"   Accuracy: {test_acc*100:.2f}%")

# Confusion Matrix
cm = confusion_matrix(labels, preds)
print(f"\n📊 Confusion Matrix:")
print(f"\nLabels: {list(id2label.values())}")
print(cm)


In [ ]:
# saving

model_path = '/content/drive/MyDrive/emotion_project/mh_model_final'
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

import json
with open(f'{model_path}/label_mapping.json', 'w') as f:
    json.dump({'label2id': label2id, 'id2label': id2label}, f, indent=2)

print(f"\n✅ Saved to: {model_path}")
print("\n🎉 MODEL 3 COMPLETE!")
print("\n" + "="*60)
print("🏆 ALL 3 MODELS TRAINED!")
print("="*60)